### Imports 

In [4]:
import gurobipy as gp

In [7]:
class OptimizationModel:
    def __init__(self, C_T, C_D, D, N, h, d_t, d_d, t_ij, t_0i, Q, Q_bar, T, T_prime):
        self.C_T = C_T #truck transportation cost
        self.C_D = C_D #drone transportation cost
        self.D = D #number of drones
        self.N = N #number of clients 
        self.h = h
        self.d_t = d_t #truck distance matrix
        self.d_d = d_d #drone distances from depot vector
        self.t_ij = t_ij
        self.t_0i = t_0i
        self.Q = Q
        self.Q_bar = Q_bar
        self.T = T
        self.T_prime = T_prime
        
        self.model = gp.Model("OptimizationModel")
        self.x = None
        self.y = None
        self.z = None
        self.u = None
        
    def build_model(self):
        # Define the decision variables
        self.x = self.model.addVars([(i,j) for i in range(self.N + 1) for j in range(self.N +1)], vtype=gb.GRB.BINARY, name="x")#ho messo +1 perchè lo 0 è il depot e poi ci sono N clienti
        self.y = self.model.addVars([(i,k) for i in range(1, self.N + 1) for k in range(self.D)], vtype=gb.GRB.BINARY, name="y")#ok
        self.z = self.model.addVars([(i,j) for i in range(self.N + 1) for j in range(self.N +1)], '''lb=0, ub=self.T,''' vtype=gb.GRB.CONTINUOUS, name="z")#ok (lb e ub sono già specificati nei constraits)
        self.u = self.model.addVars([(i) for i in range(1, self.N + 1)], '''lb=0, ub=self.Q''', vtype=gb.GRB.CONTINUOUS, name="u")#ok

        # Define the objective function
        self.model.setObjective(
            gp.quicksum(self.C_T * self.d_t[i, j] * self.x[i, j] for i in range (self.N +1) for j in range (self.N+1) if i != j) +
            gp.quicksum(self.C_D * self.d_d[k] * self.y[k, l] for k in range (self.N+1) for l in self.D),
            gb.GRB.MINIMIZE
        )

        # Aggiungere i vincoli
        self.add_constraints()

    def add_constraints(self):
        self.model.addConstr(gp.quicksum(self.x[0, i] for i in self.C) <= self.h, "Vincolo(2)")

        for j in self.N:
            self.model.addConstr(gp.quicksum(self.x[i, j] for i in self.N if i != j) == gp.quicksum(self.x[j, k] for k in self.N if k != j), "Vincolo(3)")

        for j in self.C_T:
            self.model.addConstr(gp.quicksum(self.x[i, j] for i in self.N if i != j) + gp.quicksum(self.y[k, j] for k in self.D) == 1, "Vincolo(4)")

        for j in self.C_T:
            self.model.addConstr(gp.quicksum(self.x[i, j] for i in self.N if i != j) == 1, "Vincolo(5)")

        for i in self.C:
            for j in self.C:
                if j != i:
                    self.model.addConstr(self.u[i] - self.u[j] + self.Q * self.x[i, j] <= self.Q - self.Q_bar[j], "Vincolo(6)")

        for k in self.D:
            self.model.addConstr(gp.quicksum(self.y[k, j] * self.t_ij[j, k] for j in self.C_T) <= self.T_prime, "Vincolo(7)")

        for i in self.C:
            self.model.addConstr(self.z[i, self.h] + gp.quicksum(self.t_ij[i, j] * self.x[i, j] for j in self.N if j != i) == gp.quicksum(self.z[j, i] for j in self.N if j != i), "Vincolo(8)")

        for i in self.C:
            self.model.addConstr(self.z[0, i] == self.t_0i * self.x[0, i], "Vincolo(9)")

        for i in self.C:
            self.model.addConstr(self.z[i, 0] <= self.T * self.x[i, 0], "Vincolo(10)")

        self.model.addConstrs((self.x[i, j] == 0 for i in self.N for j in self.N if i == j), "Vincolo(11)")
        self.model.addConstrs((self.y[j, k] == 0 for j in self.C_T for k in self.D), "Vincolo(12)")
        self.model.addConstrs((0 <= self.u[i] <= self.Q for i in self.C), "Vincolo(13)")
        self.model.addConstrs((0 <= self.z[i, j] <= self.T for i in self.N for j in self.N if i != j), "Vincolo(14)")

    def solve(self):
        self.model.optimize()

    def print_results(self):
        if self.model.status == GRB.OPTIMAL:
            for v in self.model.getVars():
                print(f'{v.varName}: {v.x}')
            print(f'Obj: {self.model.objVal}')
        else:
            print("No optimal solution found.")

'''
# Esempio di utilizzo della classe
# Definire i dati del problema (questo deve essere adattato ai dati specifici)
C = ...
C_T = ...
D = ...
N = ...
h = ...
d_ij = ...
d_prime_jk = ...
t_ij = ...
t_0i = ...
Q = ...
Q_bar = ...
T = ...
T_prime = ...

# Creare un'istanza della classe
opt_model = OptimizationModel(C, C_T, D, N, h, d_ij, d_prime_jk, t_ij, t_0i, Q, Q_bar, T, T_prime)

# Costruire il modello
opt_model.build_model()

# Risolvere il modello
opt_model.solve()

# Stampare i risultati
opt_model.print_results()
'''

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1029223463.py, line 27)